<a href="https://colab.research.google.com/github/LarrySnyder/stockpyl/blob/master/notebooks/with_solutions/Stockpyl_Tutorial_%C2%A73_Supply_Uncertainty_WITH_SOLUTIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stockpyl Tutorial
=================

(This notebook is a companion to Snyder, L. V., "[Stockpyl: A Python Package for Inventory Optimization and Simulation](https://pubsonline.informs.org/doi/10.1287/educ.2023.0256)," in: Bish, E. K. and H. Balasubramanian, _INFORMS TutORials in Operations Research_, 156–197, 2023.)



# Section 3: Supply Uncertainty

The literature on inventory optimization with supply uncertainty considers two main types of supply uncertainty: disruptions (in which the supplier is unavailable for a random period of time) and yield uncertainty (in which the quantity delivered differs randomly from the quantity ordered). (Other forms of supply uncertainty, such as lead time uncertainty or capacity uncertainty, are somewhat less well-studied in the literature, and are not implemented in Stockpyl.)
In particular, the `supply_uncertainty` module in Stockpyl implements the following:

* The economic order quantity (EOQ) model with disruptions (Section 3.1.1)
* The economic order quantity (EOQ) model with yield uncertainty (Section 3.1.2)
* The newsvendor model with disruptions (Section 3.2.1)
* The newsvendor with yield uncertainty (Section 3.2.2)

The models implemented in Stockpyl (and most inventory models with disruptions) treat disruptions as binary events---either the supplier is completely disrupted and cannot supply any items, or it is completely operational and has infinite supply available. The supplier's status is governed by a 2-state Markov chain, with the `UP` and `DOWN` states indicating that the supplier is operational or disrupted, respectively. For continuous-review inventory models (EOQ), we use a continuous-time Markov chain (CTMC) with given transition rates, and for periodic-review models (newsvendor), we use a discrete-time Markov chain (DTMC) with given transition probabilties. These are discussed in more detail in Sections 3.1.1 and 3.2.1. For reviews of inventory models with disruptions, see Atan and Snyder (2012) or Snyder, et al. (2016).

In contrast, yield uncertainty assumes that the supplier is always operational, but the quantity delivered differs from the quantity ordered by a random amount. Stockpyl handles two common types of yield uncertainty, additive and multiplicative. In additive yield uncertainty, if we order $Q$ units, we receive $Q+Y$, where $Y$ is a random variable. In multiplicative yield uncertainty, we receive $QZ$ units, where $Z$ is a random variable. In both cases, the probability distribution of the yield ($Y$ or $Z$) is assumed to be known, analogous to the case of demand uncertainty. Sometimes, restrictions are placed on the yield random variables; for example, we might assume $Y \le 0$ or $Z \le 1$ to ensure that the quantity received is not greater than the quantity ordered. (This is realistic in some settings but not in others; for example, in agriculture or chemical manufacturing, the yield is sometimes larger than intended.) However, Stockpyl does not impose such restrictions; if restrictions are required, they should be incorporated into the distribution information passed to the functions. For reviews of inventory models with yield uncertainty, see Grosfeld Nir and Gerchak (2004) or Yano and Lee (1995).



First, install the package:

In [1]:
!pip install stockpyl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.3/146.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.4/526.4 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.5/398.5 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 30.1 MB/s eta 0:00:00
INFO: pip is lo

## 3.1 The EOQ Problem With Supply Uncertainty



### 3.1.1 The EOQ Problem with Disruptions (EOQD)

The EOQ problem with disruptions (EOQD; _FoSCT_ §9.2.1) assumes that the supplier can be disrupted randomly, according to a 2-state CTMC. The CTMC transitions from the `UP` state to the `DOWN` state with rate $\lambda$, known as the _disruption rate_, and from the `DOWN` state to the `UP` state with rate $\mu$, the _recovery rate_. Therefore, the length of a given `UP` (non-disrupted) interval is exponentially distributed with a mean of $1/\lambda$, and the length of a given `DOWN` (disrupted) interval is exponentially distributed with a mean of $1/\mu$. Demand is constant and deterministic, as in the classical EOQ. Inventory may be depleted during a `DOWN` interval, in which case unmet demands are lost.

The  [`eoq_with_disruptions()`](https://stockpyl.readthedocs.io/en/latest/api/seio/supply_uncertainty.html#stockpyl.supply_uncertainty.eoq_with_disruptions) function solves the EOQD either exactly (using models by Parlar and Berkin (1991) and Berk and Arreola-Risa (1994) and optimizing numerically), or heuristically (using an approximation by Snyder (2014)).


The [`eoq_with_disruptions()`](https://stockpyl.readthedocs.io/en/latest/api/seio/supply_uncertainty.html#stockpyl.supply_uncertainty.eoq_with_disruptions) function has the signature
```python
eoq_with_disruptions(fixed_cost, holding_cost, stockout_cost, demand_rate,
    disruption_rate, recovery_rate, approximate=False)
```
and returns two parameters: `order_quantity` and `cost`. The function numerically optimizes the exact cost function if `approximate=False` and the approximation cost function if `approximate=True`.

**Example:** Suppose the fixed cost is 8, the holding cost is 0.225, the stockout cost is 5, the demand rate is 1300, and the disruption and recovery rates are 1.5 and 14, respectively (_FoSCT_ Examples 9.1–9.2). Solving the problem exactly, we get $Q^* = 772.81$, with an expected cost of 173.95:

In [2]:
from stockpyl.supply_uncertainty import eoq_with_disruptions
eoq_with_disruptions(8, 0.225, 5, 1300, 1.5, 14)

(772.8110739983106, 173.95000257319708)

If we instead use the approximation, we get a very close order quantity, $Q=773.14$:

In [3]:
eoq_with_disruptions(8, 0.225, 5, 1300, 1.5, 14, approximate=True)

(773.1432417118889, 173.957229385175)

The cost returned by `eoq_with_disruptions()` when `approximate=True` is an approximate cost. We can calculate the exact cost of the approximate order quantity using the [`eoq_with_disruptions_cost()`](https://stockpyl.readthedocs.io/en/latest/api/seio/supply_uncertainty.html#stockpyl.supply_uncertainty.eoq_with_disruptions_cost) function, which takes the order quantity as an input parameter and returns the (exact or approximate) cost. In this case, the exact and approximate costs are nearly identical:

In [4]:
from stockpyl.supply_uncertainty import eoq_with_disruptions_cost
eoq_with_disruptions_cost(773.1432417118889, 8, 0.225, 5, 1300, 1.5, 14, approximate=False)

173.95001838749064

---
**Exercise:** Solve the EOQD with $K=17$, $h=4$, $p=60$, a demand rate of 9, and disruption and recovery rates 2 and 12, respectively. Use both the exact and approximate models, and compare their solutions and costs.

In [5]:
## SOLUTION
eoq_with_disruptions(17, 4, 60, 9, 2, 12, approximate=False)

(10.16125677849806, 40.645030286768154)

In [6]:
## SOLUTION
eoq_with_disruptions(17, 4, 60, 9, 2, 12, approximate=True)

(10.161257749592748, 40.64503099837099)

In [7]:
## SOLUTION
eoq_with_disruptions_cost(10.161257749592748, 17, 4, 60, 9, 2, 12, approximate=False)

40.645030286769114

### 3.1.2 The EOQ Problem with Yield Uncertainty

In the EOQ problem with yield uncertainty (_FoSCT_ §9.3.1), an order of size $Q$ is placed, and the quantity received is $Q+Y$ (in the case of additive yield) or $QZ$ (in the case of multiplicative yield), where $Y$ and $Z$ are random variables.  The lead time is 0 and stockouts are not allowed.

> **Remark:** The expected cost functions and the optimal solutions for the EOQ problems with additive and multiplicative yield uncertainty depend only on the mean and standard deviation of the yield random variable, not the entire distribution. Therefore, the corresponding functions only require these quantities, not an entire distribution function, and they make no assumptions about the shape of the distribution.

The Stockpyl functions for EOQ with yield uncertainty have the signatures
```python
eoq_with_additive_yield_uncertainty(fixed_cost, holding_cost, demand_rate,
	yield_mean, yield_sd, order_quantity=None)
eoq_with_multiplicative_yield_uncertainty(fixed_cost, holding_cost,
	demand_rate, yield_mean, yield_sd, order_quantity=None)
```
Each function returns two parameters: `order_quantity` and `cost`. The optimal input parameter `order_quantity` allows you to provide the order quantity, in which case the function will simply calculate the cost for that order quantity.


**Example:** Suppose the fixed cost is 18500, the holding cost is 0.06 per item per month, the demand rate is 75000 per month, and there is additive yield uncertainty with $Y\sim N(-15000,9000^2)$ (_FoSCT_ Example 9.4). That is, the yield is, on average, 15000 units less than $Q$, with a standard deviation of 9000. Solving the problem optimally, we get an optimal order quantity of 230246.37, with an opimal expected cost of 12914.78 per month:

In [9]:
from stockpyl.supply_uncertainty import eoq_with_additive_yield_uncertainty
import math
eoq_with_additive_yield_uncertainty(18500, 0.06, 75000, -15000, 9000)

(230246.37046881882, 12914.78222812913)

We can also provide the function with an order quantity in order to calculate the cost of that quantity (without optimizing):

In [10]:
eoq_with_additive_yield_uncertainty(18500, 0.06, 75000, -15000, 9000, order_quantity=300000)

(300000, 13426.947368421053)

**Example:** Suppose instead that the yield uncertainty is multiplicative, with $Z\sim \text{Beta}(5, 1)$; that is, the yield multiplier has a beta distribution with parameters 5 and 1 (Example 9.5). Then the optimal order quantity is 254477.46, with an expected cost per month of 13086.16:

In [15]:
from scipy.stats import beta
# First, calculate the mean and standard deviation of a Beta(5, 1) r.v.
mu, sigma_squared = beta.stats(5, 1, moments='mv')
sigma = math.sqrt(sigma_squared)
mu, sigma

(0.8333333333333334, 0.14085904245475275)

In [17]:
from stockpyl.supply_uncertainty import eoq_with_multiplicative_yield_uncertainty
# Now solve the EOQ with yield uncertainty.
eoq_with_multiplicative_yield_uncertainty(18500, 0.06, 75000, mu, sigma)

(254460.21299998942, 13086.525239999457)

---
**Exercise:** Suppose the fixed cost is 1500, the holding cost is 3 per item per week, the demand rate is 2000 per week, and there is additive yield uncertainty with $Y\sim U[-50,20]$. Find the optimal order quantity and cost.

In [21]:
## SOLUTION
# Mean and variance of a U[a,b] rv are (a+b)/2 and (b-a)^2/12.
a = -50
b = 20
mu = (a + b) / 2
sigma = math.sqrt((b - a)**2 / 12)
eoq_with_additive_yield_uncertainty(1500, 3, 2000, mu, sigma)

(1429.3579226395748, 4243.073767918724)

**Exercise:** Suppose instead that the yield uncertainty is multiplicative, with $Z\sim N(0.9, 0.02^2)$.

In [22]:
## SOLUTION
eoq_with_multiplicative_yield_uncertainty(1500, 3, 2000, 0.9, 0.02)

(1570.9605590167373, 4243.688123423879)

## 3.2 The Newsvendor Problem with Supply Uncertainty

We now turn our attention to newsvendor problems with supply uncertainty. These problems assume that the demand is deterministic but the supply is stochastic (the opposite of the classical newsvendor problem). We consider an infinite-horizon model in which inventory may be carried from one time period to the next (at a holding cost of $h$ per item per period), and unmet demands may be carried from one period to the next in the form of backorders (at a stockout cost of $p$ per item per period).

> **Remark:** The newsvendor problem with disruptions is an infinite-horizon problem; it does not make sense to consider this problem in a single-period setting since the only reason to order extra inventory in a given period is to protect against disruptions in future periods. The newsvendor problem with yield uncertainty may be interpreted as either a single-period or infinite-horizon problem. As discussed in Section 1, we use the term "newsvendor" to describe both the single-period and infinite-horizon problems.

### 3.2.1 The Newsvendor Problem with Disruptions

The newsvendor problem with disruptions (Güllu, Onol, and Erkip 1997; Tomlin 2005 [unabridged version]; Schmitt and Snyder 2012; _FoSCT_ §9.2.2)  assumes that the supplier's disruptions follow a 2-state DTMC. If the supplier is in the `UP` state, it transitions to the `DOWN` state with probability $\alpha$ (called the _disruption probability_) and remains `UP` with probability $1-\alpha$. If it is in the `DOWN` state, it transitions to `UP` with probability $\beta$ (the _recovery probability_) and remains `DOWN` with probability $1-\beta$. Therefore, the length of a given `UP` (non-disrupted) interval has a geometric distribution with mean $1/\alpha$, and the length of a given `DOWN` (disrupted) interval is geometrically distributed with mean $1/\beta$. Inventory may be depleted during a `DOWN` interval, in which case unmet demands are lost. A base-stock policy is optimal (with the modification that the firm orders nothing during `DOWN` periods), and our goal is to determine the optimal base-stock level.

The [`newsvendor_with_disruptions()`](https://stockpyl.readthedocs.io/en/latest/api/seio/supply_uncertainty.html#stockpyl.supply_uncertainty.newsvendor_with_disruptions) function has the signature
```python
newsvendor_with_disruptions(holding_cost, stockout_cost, demand,
    disruption_prob, recovery_prob, base_stock_level=None)
```
and returns two parameters: `base_stock_level` and `cost`. Alternatively, you can use the optional input parameter `base_stock_level` to specify a base-stock level, for which the function will calculate the expected cost.

**Example:** Suppose the holding and stockout costs are 0.25 and 3 per item per period, respectively; the demand is 2000 items per period; and the disruption and recovery probabilities are 0.04 and 0.25, respectively (_FoSCT_ Example 9.3). Then the optimal base-stock level is 8000, and the corresponding expected cost per period is 2737.07:

In [23]:
from stockpyl.supply_uncertainty import newsvendor_with_disruptions
newsvendor_with_disruptions(0.25, 3, 2000, 0.04, 0.25)

(8000, 2737.0689302470355)

---
**Exercise:** Suppose the holding and stockout costs are 35 and 600 per item per period, respectively; the demand is 50 items per period; and the disruption and recovery probabilities are 0.1 and 0.3, respectively. Find the optimal base-stock level and expected cost.

In [25]:
## SOLUTION
newsvendor_with_disruptions(35, 600, 50, 0.1, 0.3)

(300, 11738.518591751188)

### 3.2.2 The Newsvendor Problem with Yield Uncertainty

The last model with supply uncertainty we discuss is the newsvendor problem with yield uncertainty (_FoSCT_ §9.3.2). As of this writing, Stockpyl only supports additive yield uncertainty for the newsvendor problem.

The [`newsvendor_with_additive_yield_uncertainty()`](https://stockpyl.readthedocs.io/en/latest/api/seio/supply_uncertainty.html#stockpyl.supply_uncertainty.newsvendor_with_additive_yield_uncertainty) function has the signature
```python
newsvendor_with_additive_yield_uncertainty(holding_cost, stockout_cost,
	demand, yield_mean=None, yield_sd=None, yield_distribution=None,
	loss_function=None, base_stock_level=None)
```
and returns two parameters: `base_stock_level` and `cost`. Alternatively, you can use the optional input parameter `base_stock_level` to provide a base-stock level, for which the function will calculate the expected cost.

This function provides two ways to specify the yield distribution:
* Using `yield_mean` and `yield_sd`, in which case the yield is assumed to be normally distributed with these parameters
* Using `yield_distribution` as a [`scipy.stats.rv_continuous`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html) or [`scipy.stats.rv_discrete`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html) object

The cost function requires the loss functions $n_Y(\cdot)$ and $\bar{n}_Y(\cdot)$ (see _FoSCT_ §C.3). If `yield_distribution` is provided, the function will calculate the loss functions using generic formulas.
However, you can optionally also provide a `loss_function` to be used instead. If `loss_function` is provided, it must be a function that takes a single argument and returns a tuple consisting of the loss function and the complementary loss function value of that argument. (Stockpyl's `loss_function` module contains a number of functions for calculating loss functions.) The `loss_function` parameter is ignored if `yield_distribution` is not provided. Providing the loss function explicitly is usually more accurate.



**Example:** Suppose the holding cost is 15 per item per period, the stockout cost is 75 per item per period, the demand is 150000, and the yield uncertainty is additive with $Y\sim U[-500000, 500000]$ (_FoSCT_ Example 9.6). We first solve it without providing the loss functions explicitly (so they are calculated using a generic function):

In [27]:
from stockpyl.supply_uncertainty import newsvendor_with_additive_yield_uncertainty
from scipy.stats import uniform
# Second parameter scipy.stats.uniform is the range, not the upper limit.
yield_distribution = uniform(-5e5, 1e6)
newsvendor_with_additive_yield_uncertainty(15, 75, 1.5e6, yield_distribution=yield_distribution)

(1833333.3333333335, 6249999.997499999)

Next, we explicitly pass the `uniform_loss` function to calculate the loss functions, which is more accurate:

In [28]:
from stockpyl.loss_functions import uniform_loss
loss_function = lambda x: uniform_loss(x, -500000, 500000)
newsvendor_with_additive_yield_uncertainty(15, 75, 1.5e6,
    yield_distribution=uniform(-500000, 1000000),
    loss_function=loss_function)

(1833333.3333333335, 6250000.000000001)

---
**Exercise:** Suppose the holding cost is 0.6 per item per period, the stockout cost is 3.2 per item per period, the demand is 135, and the yield uncertainty is additive with $Y\sim U[-8, 6]$. Find the optimal base-stock level and expected cost, either using a generic loss function or providing one explicitly.

In [29]:
## SOLUTION
yield_distribution = uniform(-8, 14)
newsvendor_with_additive_yield_uncertainty(0.6, 3.2, 135, yield_distribution=yield_distribution)

(140.78947368421052, 3.5368421038484206)

**Next Up:** Stockpyl Tutorial §4: Multi-Echelon Inventory Optimization (MEIO)